In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
f_fire=pd.read_csv("forestfires.csv")
f_fire

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,0,1,0,0,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,0,0,0,0,0,1,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,0,0,0,0,0,1,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,1,0,0,0,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,1,0,0,0,0,small
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,...,0,0,0,0,0,0,0,0,0,large
513,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,...,0,0,0,0,0,0,0,0,0,large
514,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,...,0,0,0,0,0,0,0,0,0,large
515,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,...,0,0,0,0,0,0,0,0,0,small


In [3]:
f_fire.isna().sum()

month            0
day              0
FFMC             0
DMC              0
DC               0
ISI              0
temp             0
RH               0
wind             0
rain             0
area             0
dayfri           0
daymon           0
daysat           0
daysun           0
daythu           0
daytue           0
daywed           0
monthapr         0
monthaug         0
monthdec         0
monthfeb         0
monthjan         0
monthjul         0
monthjun         0
monthmar         0
monthmay         0
monthnov         0
monthoct         0
monthsep         0
size_category    0
dtype: int64

In [4]:
f_fire[f_fire.duplicated()].shape

(8, 31)

In [5]:
# lets drop the duplicate values
f_fire.drop_duplicates().reset_index(drop=True)
f_fire

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,0,1,0,0,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,0,0,0,0,0,1,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,0,0,0,0,0,1,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,1,0,0,0,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,1,0,0,0,0,small
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,...,0,0,0,0,0,0,0,0,0,large
513,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,...,0,0,0,0,0,0,0,0,0,large
514,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,...,0,0,0,0,0,0,0,0,0,large
515,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,...,0,0,0,0,0,0,0,0,0,small


In [6]:
# lets convert the cogertorical data to labels
lm=LabelEncoder()
f_fire['month']=lm.fit_transform(f_fire['month'])
f_fire['day']=lm.fit_transform(f_fire['day'])
f_fire['size_category']=lm.fit_transform(f_fire['size_category'])
f_fire

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,7,0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,0,1,0,0,0,0,1
1,10,5,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,0,0,0,0,0,1,0,1
2,10,2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,0,0,0,0,0,1,0,1
3,7,0,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,1,0,0,0,0,1
4,7,3,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,1,3,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,...,0,0,0,0,0,0,0,0,0,0
513,1,3,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,...,0,0,0,0,0,0,0,0,0,0
514,1,3,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,...,0,0,0,0,0,0,0,0,0,0
515,1,2,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,...,0,0,0,0,0,0,0,0,0,1


In [7]:
#since the month column is represented in the form of onehot coding lets drop the month column
f_fire1=f_fire.drop(['month'],axis=1)
f_fire1

,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00,...,0,0,0,0,1,0,0,0,0,1
1,5,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00,...,0,0,0,0,0,0,0,1,0,1
2,2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00,...,0,0,0,0,0,0,0,1,0,1
3,0,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00,...,0,0,0,0,1,0,0,0,0,1
4,3,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,3,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44,...,0,0,0,0,0,0,0,0,0,0
513,3,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29,...,0,0,0,0,0,0,0,0,0,0
514,3,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16,...,0,0,0,0,0,0,0,0,0,0
515,2,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00,...,0,0,0,0,0,0,0,0,0,1


In [8]:
# lets spilt it into the two feature and target variables
X=f_fire1.iloc[:,0:29]
Y=f_fire1.iloc[:,-1]

In [9]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.33,random_state=10)

In [10]:
model_f_fire=SVC(kernel='rbf')
model_f_fire.fit(x_train,y_train)
y_pred_fire=model_f_fire.predict(x_test)
acc=accuracy_score(y_test,y_pred_fire) *100
print('Accuarcy score for rbf kernel',acc)
print(classification_report(y_pred_fire,y_test))

Accuarcy score for rbf kernel 73.68421052631578
              precision    recall  f1-score   support

           0       0.02      1.00      0.04         1
           1       1.00      0.74      0.85       170

    accuracy                           0.74       171
   macro avg       0.51      0.87      0.45       171
weighted avg       0.99      0.74      0.84       171



In [11]:
confusion_matrix(y_test,y_pred_fire)

array([[  1,  45],
       [  0, 125]], dtype=int64)

LETS DO GRID SEARCH CV

In [40]:
model_fire=SVC()
param_grid=[{'kernel':['rbf'],'gamma':[50,5,10,0.5,0.001],'C':[1,11,12,13,14,15,0.1]}]
gsv=GridSearchCV(model_fire,param_grid=param_grid,cv=10)
gsv.fit(x_train,y_train)


GridSearchCV(cv=10, estimator=SVC(),
             param_grid=[{'C': [1, 11, 12, 13, 14, 15, 0.1],
                          'gamma': [50, 5, 10, 0.5, 0.001],
                          'kernel': ['rbf']}])

In [41]:
gsv.best_params_,gsv.best_score_

({'C': 11, 'gamma': 0.001, 'kernel': 'rbf'}, 0.8465546218487395)

In [43]:
# for these values lets find the accuarcy and cu=onfusion matrix and report
model_fire=SVC(kernel='rbf',C=11,gamma=0.001)
model_fire.fit(x_train,y_train)
y_pred_grid=model_fire.predict(x_test)
acc=accuracy_score(y_test,y_pred_grid)
print('accuracy score for thr best C and gamma value',acc)
print(classification_report(y_pred_grid,y_test))

accuracy score for thr best C and gamma value 0.8888888888888888
              precision    recall  f1-score   support

           0       0.65      0.91      0.76        33
           1       0.98      0.88      0.93       138

    accuracy                           0.89       171
   macro avg       0.81      0.90      0.84       171
weighted avg       0.91      0.89      0.90       171



In [44]:
confusion_matrix(y_test,y_pred_grid)

array([[ 30,  16],
       [  3, 122]], dtype=int64)